## A/B/n Tests Result Analyzer

### Notebook Overview

1. **Reading data from the BigQuery database**:
In this section, we will retrieve the required data from the BigQuery database and load it into the notebook for further analysis.

2. **Preprocessing data to get explanatory data frame**:
Before applying any statistical algorithms, it's essential to preprocess the data to get it into the right format. We will clean, transform, and organize the data to create an explanatory data frame.

3. **Developing and applying an inferential statistics algorithm to the tests' data**:
In this part, we will develop and apply the inferential statistics algorithm to analyze the tests' data. This step will help us draw meaningful insights and make inferences from the data.

4. **Returning results in a data frame form to the BigQuery database**:
After performing the statistical analysis, we will format and structure the results into a data frame. Then, we will return the results to the BigQuery database for further use or reporting.

In [ ]:
# libraries

import os
import pandas as pd
import numpy as np
import scipy.stats as st
import statsmodels.stats.power as smp
from statsmodels.stats.power import FTestAnovaPower
from scipy.stats import norm
from google.cloud import bigquery
from google.oauth2 import service_account
from pandas_gbq import to_gbq
import warnings

In [ ]:
# reading from bigquery

def connector(credentials_i, project_i, query_i):
    """
    This function returns the query output from 'clockify-analytics' project in BigQuery.
    """
    try:
        credentials=service_account.Credentials.from_service_account_file(credentials_i)
        client=bigquery.Client(credentials=credentials, project=project_i)
        query_job=client.query(query_i)
        return(query_job.result().to_dataframe())
    except ErrorValues:
        return None

query = (
"""WITH google_analytics AS (
   SELECT
     CASE
        WHEN SUBSTR(usepro.key, 35) = 'WclobaADTXOQOtKhWEy-iA' THEN 'Blog - Sticky Header'
        WHEN SUBSTR(usepro.key, 35) = 'yrYnhIfsSNiRlMa7BQIG2g' THEN 'Testing Split URL'
        WHEN SUBSTR(usepro.key, 35) = 'Jneh4c-8SvmwCcJ0n8sJyw' THEN 'A/A test 1'
        WHEN SUBSTR(usepro.key, 35) = 'gx7P61YUR--ATNBoEGMQIg' THEN 'Goal to link Ration 1:1'
        WHEN SUBSTR(usepro.key, 35) = 'qjQilm6yRWSgivobFJEOfQ' THEN 'Trial section on pricing'
        WHEN SUBSTR(usepro.key, 35) = 'vaqi_KkERUGZzuDYwoeWdA' THEN 'Shorter Copy test'
        WHEN SUBSTR(usepro.key, 35) = 'rTtefV7LTiegat4tczi5zg' THEN 'Pricing link in the header PPC'
        WHEN SUBSTR(usepro.key, 35) = 'nRYKJ1ksTPqpefkPIE6gAA' THEN 'PPC timesheet - GTL 1:1'
        ELSE SUBSTR(usepro.key, 35)
    END AS experiment,
    CASE
       WHEN usepro.value.string_value LIKE '%.0' THEN 'Control Group'
       WHEN usepro.value.string_value LIKE '%.1' THEN 'Alternative Group'
    END AS variant,
    CASE
       WHEN event_name IN ('sign_up', 'purchase', 'free_trial') THEN event_name
    END AS event_name,
    user_pseudo_id as user_pseudo_id,
    CASE
       WHEN user_id IS NOT NULL THEN user_id
       WHEN usepro.key = 'user_id' AND usepro.value.string_value != '[user_id]' OR usepro.value.string_value IS NOT NULL THEN usepro.value.string_value
    END AS user_id,
    parse_date('%Y%m%d', event_date) as event_date,
    row_number() over (order by event_date desc) as rnum
  FROM `clockify-analytics.analytics_290057802.events_*`,
  UNNEST(user_properties) AS usepro
  WHERE usepro.key LIKE "%belongs_to_optimize_experiment_id%" AND event_name IN ('sign_up', 'purchase', 'free_trial')
)

SELECT
    a.experiment,
    a.variant,
    a.event_name,
    a.user_pseudo_id,
    a.user_id,
    b.customer_id,
    a.event_date
FROM google_analytics a
left join `clockify-analytics.analytics_290057802.stripe_customer_table` b on a.user_id = b.user_id
left join `clockify-analytics.analytics_290057802.stripe_payments_table` p on p.customer_id = b.customer_id;
"""
)

df=connector(os.environ.get('credentials_clockify'), os.environ.get('project_clockify'), query)

In [ ]:
df.tail()

,experiment,variant,event_name,user_pseudo_id,user_id,customer_id,event_date
11896,Pricing link in the header PPC,Control Group,purchase,732914387.1677179465,63f7ba6b5db8ee16dc805a75,cus_NPZLykMkklh2rG,2023-03-31
11897,Pricing link in the header PPC,Control Group,purchase,732914387.1677179465,63f7ba6b5db8ee16dc805a75,cus_NPZLykMkklh2rG,2023-03-31
11898,Pricing link in the header PPC,Control Group,purchase,732914387.1677179465,63f7ba6b5db8ee16dc805a75,cus_NPZLykMkklh2rG,2023-03-31
11899,Pricing link in the header PPC,Control Group,purchase,732914387.1677179465,63f7ba6b5db8ee16dc805a75,cus_NPZLykMkklh2rG,2023-03-31
11900,Pricing link in the header PPC,Control Group,purchase,732914387.1677179465,63f7ba6b5db8ee16dc805a75,cus_NPZLykMkklh2rG,2023-03-31


In [ ]:
df

,experiment,variant,event_name,user_pseudo_id,user_id,customer_id,event_date
0,Pricing link in the header PPC,Control Group,purchase,311305085.1680346493,6427fcc9536c7900917d2ea5,cus_NdHXmRH0zIIqXt,2023-04-01
1,Pricing link in the header PPC,Control Group,purchase,311305085.1680346493,6427fcc9536c7900917d2ea5,cus_NdHXmRH0zIIqXt,2023-04-01
2,Pricing link in the header PPC,Control Group,purchase,311305085.1680346493,6427fcc9536c7900917d2ea5,cus_NdHXmRH0zIIqXt,2023-04-01
3,Pricing link in the header PPC,Control Group,purchase,311305085.1680346493,6427fcc9536c7900917d2ea5,cus_NdHXmRH0zIIqXt,2023-04-01
4,Pricing link in the header PPC,Control Group,purchase,311305085.1680346493,6427fcc9536c7900917d2ea5,cus_NdHXmRH0zIIqXt,2023-04-01
...,...,...,...,...,...,...,...
11896,Pricing link in the header PPC,Control Group,purchase,732914387.1677179465,63f7ba6b5db8ee16dc805a75,cus_NPZLykMkklh2rG,2023-03-31
11897,Pricing link in the header PPC,Control Group,purchase,732914387.1677179465,63f7ba6b5db8ee16dc805a75,cus_NPZLykMkklh2rG,2023-03-31
11898,Pricing link in the header PPC,Control Group,purchase,732914387.1677179465,63f7ba6b5db8ee16dc805a75,cus_NPZLykMkklh2rG,2023-03-31
11899,Pricing link in the header PPC,Control Group,purchase,732914387.1677179465,63f7ba6b5db8ee16dc805a75,cus_NPZLykMkklh2rG,2023-03-31


In [ ]:
# EDA

eda=df.copy()
eda.columns=['experiments','variants','target_events','participants','signupers','purchases','dates']
eda.describe().drop(['count','top','freq'])

,experiments,variants,target_events,participants,signupers,purchases,dates
unique,7,2,3,2612,145,169,71


In [ ]:
df.columns

Index(['experiment', 'variant', 'event_name', 'user_pseudo_id', 'user_id',
       'customer_id', 'event_date'],
      dtype='object')

In [ ]:
# data preprocessing

def preprocessing(sample):
    """
    This function aggregates key metrics by experiment and variant.
    Key metrics included are: 'sample_size', 'signups', 'free_trials', and 'purchases'.
    """
    df_final=sample.groupby(['experiment','variant','event_date']).agg(
    sample_size=('user_pseudo_id', lambda x: x.nunique()),
    signups=('user_id', lambda x: x.nunique()),
    free_trials=('user_id', lambda x: x[sample['event_name'] == 'free_trial'].nunique()),
    purchases=('user_id', lambda x: x[sample['event_name']=='purchase'].nunique())
    ).reset_index()

    return df_final

df_ab=preprocessing(df)

df_ab['signups_cvr'] = round(df_ab['signups'] / df_ab['sample_size'],2)
df_ab['trials_cvr'] = round(df_ab['free_trials'] / df_ab['sample_size'],2)
df_ab['purchases_cvr'] = round(df_ab['purchases'] / df_ab['sample_size'],2)

df_ab=df_ab[['experiment','variant','event_date','sample_size','signups','signups_cvr','free_trials','trials_cvr','purchases','purchases_cvr']]

df_ab

,experiment,variant,event_date,sample_size,signups,signups_cvr,free_trials,trials_cvr,purchases,purchases_cvr
0,Blog - Sticky Header,Alternative Group,2022-09-13,5,1,0.20,0,0.0,0,0.00
1,Blog - Sticky Header,Alternative Group,2022-09-14,6,1,0.17,0,0.0,0,0.00
2,Blog - Sticky Header,Alternative Group,2022-09-15,9,2,0.22,0,0.0,1,0.11
3,Blog - Sticky Header,Alternative Group,2022-09-16,8,1,0.12,0,0.0,0,0.00
4,Blog - Sticky Header,Alternative Group,2022-09-17,1,1,1.00,0,0.0,0,0.00
...,...,...,...,...,...,...,...,...,...,...
189,Trial section on pricing,Control Group,2023-03-18,1,1,1.00,0,0.0,0,0.00
190,Trial section on pricing,Control Group,2023-03-21,5,1,0.20,0,0.0,0,0.00
191,Trial section on pricing,Control Group,2023-03-22,4,3,0.75,2,0.5,0,0.00
192,Trial section on pricing,Control Group,2023-03-23,3,1,0.33,0,0.0,0,0.00


In [ ]:
def get_data_by_metric(data, metric):
    """
    Extracts and returns the data for a given metric from the provided experiment.
    """
    control_data = data[data['variant'] == 'Control Group'][metric].tolist()
    alternative_data = data[data['variant'] == 'Alternative Group'][metric].tolist()

    control_sample_size = data[data['variant'] == 'Control Group']['sample_size'].sum()
    alternative_sample_size = data[data['variant'] == 'Alternative Group']['sample_size'].sum()

    return control_data, alternative_data, control_sample_size, alternative_sample_size


In [ ]:
#data for the specified experiment and metric
experiment = df_ab[df_ab['experiment'] == 'Testing Split URL']
metric = 'purchases'
control_data, alternative_data, control_sample_size, alternative_sample_size = get_data_by_metric(experiment, metric)

print("Control Data:", control_data)
print("Alternative Data:", alternative_data)
print("Control Sample Size:", control_sample_size)
print("Alternative Sample Size:", alternative_sample_size)

Control Data: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Alternative Data: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Control Sample Size: 60
Alternative Sample Size: 29


In [ ]:
# THRESHOLD = 0.05  # or any other value < 0.05
# def check_normality(control, alternative):
#     """ Test if data is normally distributed using Shapiro-Wilk test for each group separately """

#     # Minimum number of unique values to apply Shapiro-Wilk test
#     MIN_UNIQUE_VALUES = 3

#     # Check if there's enough unique values and if the range is zero to avoid warnings
#     if (len(np.unique(control)) < MIN_UNIQUE_VALUES or np.ptp(control) == 0) or \
#        (len(np.unique(alternative)) < MIN_UNIQUE_VALUES or np.ptp(alternative) == 0):
#         return False

#     stat_control, p_control = st.shapiro(control)
#     stat_alternative, p_alternative = st.shapiro(alternative)

#     # returns True if both groups are normally distributed
#     if p_control > THRESHOLD and p_alternative > THRESHOLD:
#         return True
#     return False

In [ ]:
THRESHOLD = 0.05

def check_normality_ks(control, alternative):
    """
    Test if data is normally distributed using the Kolmogorov-Smirnov test.
    """

    def normality_test(data):
        # Check for zero variance, if 0 data not normal
        if np.std(data) == 0:
            return False

        # Standardize the data (mean 0 and variance 1) before using KS test for normality
        standardized_data = (data - np.mean(data)) / np.std(data)
        stat, p = st.kstest(standardized_data, 'norm')
        return p > THRESHOLD
    # return True when the data appears to be normally distributed
    return normality_test(control) and normality_test(alternative)

In [ ]:
def check_variance(data1, data2):
    """ Test if two groups have equal variances using Levene's test """
    stat, p = st.levene(data1, data2)
    if p > 0.05:
        return True
    return False

In [ ]:
def determine_test(control, alternative, paired=False, group_n=2):
    """
    Determine which test to apply based on data distribution, variance, and other experimental design properties.
    """
    # Check normality and variance
    is_normal = check_normality_ks(control, alternative)
    has_equal_variance = check_variance(control, alternative)

    # Decide on the appropriate test
    if is_normal and has_equal_variance:
        if group_n == 2:
            return "t-test"
        elif group_n > 2:
            return "ANOVA"
    else:
        if group_n == 2:
            if not paired:
                return "Mann-Whitney"
            else:
                return "Wilcoxon"
        elif group_n > 2:
            if not paired:
                return "Kruskal-Wallis"
            else:
                return "Friedman"

In [ ]:
def apply_test(test_name, control, alternative):
    """ Apply the determined test """
    if test_name == 't-test':
        return st.ttest_ind(control, alternative)
    elif test_name == 'Mann-Whitney':
        return st.mannwhitneyu(control, alternative, alternative='two-sided')
    elif test_name == 'Wilcoxon':
        return st.wilcoxon(control, alternative)
    elif test_name == 'ANOVA':
        return st.f_oneway(control, alternative)
    elif test_name == 'Kruskal-Wallis':
        return st.kruskal(control, alternative)
    elif test_name == 'Friedman':
        return st.friedmanchisquare(control, alternative)
    else:
        raise ValueError("Unsupported test type")

In [ ]:
alpha = 0.05
power_threshold = 0.8
def calculate_test_power(test, control, alternative, control_sample_size, alternative_sample_size):
    """ Calculate the statistical power of the test """
    control = np.array(control)
    alternative = np.array(alternative)


    # Check for zero standard deviation, avoid dividing by zero
    if control.std() == 0 or alternative.std() == 0:
        print("Warning: Standard deviation is zero. Cannot compute effect size.")
        return None

    effect_size = (control.mean() - alternative.mean()) / control.std()


    if test == "t-test":
        analysis = smp.TTestIndPower()
        return analysis.solve_power(effect_size=effect_size, nobs1=control_sample_size, ratio=alternative_sample_size / control_sample_size, alpha=alpha)

    elif test == "Mann-Whitney":
        # Rank sum for control under null hypothesis
        expected_rank_sum = control_sample_size * (control_sample_size + alternative_sample_size + 1) / 2

        # Variance of the rank sum for control under null hypothesis
        expected_variance = control_sample_size * alternative_sample_size * (control_sample_size + alternative_sample_size + 1) / 12

        # Calculate z using the expected rank sum and variance
        z = (expected_rank_sum - (control_sample_size * alternative.mean())) / np.sqrt(expected_variance)

        # Return power based on z and alpha
        return 1 - norm.cdf(z - norm.ppf(1 - alpha))

    elif test == "ANOVA":
        total_sample_size = control_sample_size + alternative_sample_size
        effect_size = (alternative.mean() - control.mean()) / control.std()
        k_groups = k_groups  # to be decided
        return calculate_anova_power(effect_size, total_sample_size, alpha, k_groups)


    elif test == "Wilcoxon":
        return simulate_power(apply_test, control_data, alternative_data, control_sample_size, alternative_sample_size, alpha)


    elif test == "Kruskal-Wallis":
        return simulate_power(apply_test, control_data, alternative_data, control_sample_size,alternative_sample_size, alpha)


    elif test == "Friedman":
        return simulate_power(apply_test, control_data, alternative_data, control_sample_size, alternative_sample_size, alpha)

In [ ]:
def calculate_anova_power(effect_size, nobs, alpha, k_groups):
    analysis = FTestAnovaPower()
    return analysis.solve_power(effect_size=effect_size, nobs=nobs, alpha=alpha, k_groups=k_groups)

def simulate_power(test_function, control, alternative, control_size, alternative_size, alpha, n_simulations=10000):
    n_rejects = 0

    for _ in range(n_simulations):
        sample_control = np.random.choice(control, control_size, replace=True)
        sample_alternative = np.random.choice(alternative, alternative_size, replace=True)

        stat, p = test_function(sample_control, sample_alternative)
        if p < alpha:
            n_rejects += 1

    power = n_rejects / n_simulations
    return power


In [ ]:
def ab_test(data):
    results = []

     # Group data by experiment
    for experiment, data in data.groupby('experiment'):
        # Check each metric separately: 'signups', 'free_trials', 'purchases'
        for metric in ['signups', 'free_trials', 'purchases']:
            control_data, alternative_data, control_sample_size, alternative_sample_size = get_data_by_metric(data, metric)

            # If both control and alternative data are zeros, skip this iteration
            if sum(control_data) == 0 and sum(alternative_data) == 0:
                continue
            if not control_data or not alternative_data:
                print(f"Missing data for experiment '{experiment}' and metric '{metric}'. Skipping...")
                continue

            # Step 1: Determine Test
            test_to_apply = determine_test(control_data, alternative_data)

            # Step 2: Apply Test
            stat, p_value = apply_test(test_to_apply, control_data, alternative_data)
            p_value = round(p_value, 2)

            # Step 3: Calculate Power
            power = calculate_test_power(test_to_apply, control_data, alternative_data, control_sample_size, alternative_sample_size)

            # Determine Outcome
            outcome = 'Alternative group demonstrated a statistically significant difference over the control.' if p_value <= alpha and (power is None or power >= power_threshold) else 'There is no statistically significant difference between the alternative and control groups.'

            results.append({
                'experiment': experiment,
                'metric': metric,
                'test_name': test_to_apply,
                'p_value' : p_value,
                'power' : power,
                'test_outcome': outcome
            })

    return pd.DataFrame(results)


In [ ]:
results = ab_test(df_ab)
print(results)

                        experiment       metric     test_name  p_value  \
0             Blog - Sticky Header      signups  Mann-Whitney     0.06   
1             Blog - Sticky Header    purchases  Mann-Whitney     0.08   
2          Goal to link Ration 1:1      signups  Mann-Whitney     0.36   
3          Goal to link Ration 1:1    purchases  Mann-Whitney     0.36   
4          PPC timesheet - GTL 1:1      signups  Mann-Whitney     0.91   
5          PPC timesheet - GTL 1:1  free_trials  Mann-Whitney     0.18   
6          PPC timesheet - GTL 1:1    purchases  Mann-Whitney     0.37   
7   Pricing link in the header PPC      signups        t-test     0.87   
8   Pricing link in the header PPC  free_trials        t-test     0.82   
9   Pricing link in the header PPC    purchases        t-test     0.92   
10               Shorter Copy test      signups  Mann-Whitney     0.05   
11               Shorter Copy test  free_trials  Mann-Whitney     0.08   
12               Shorter Copy test    

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/scipy/stats/_morestats.py:2643: RuntimeWarning: invalid value encountered in double_scalars
  W = numer / denom


In [ ]:
import warnings

with warnings.catch_warnings(record=True) as w:
    # Trigger the code that causes the warning
    warnings.simplefilter("always")  # always trigger all warnings
    results = ab_test(df_ab)

    # Display the captured warnings
    for warn in w:
        print(str(warn.message))
        print("Triggered by:", warn.category)
        print("In file:", warn.filename)
        print("On line:", warn.lineno, "\n")


invalid value encountered in double_scalars
Triggered by: <class 'RuntimeWarning'>
In file: /opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/scipy/stats/_morestats.py
On line: 2643 

